In [8]:
import networkx as nx
import numpy as np
import pandas as pd
import networkx.algorithms.community as nx_comm
import math
import matplotlib.pyplot as plt

def genetic_evolution(nodes,edges,population=15,generation=30,r=1.5):
    graph = []
    graph=nx.Graph()
    graph.add_nodes_from(nodes) #adds nodes
#     print(len(nodes))
#     print(len(edges))
    graph.add_edges_from(edges) #add edges
    Adj = nx.adjacency_matrix(graph)
    count1 = 0
    print(len(Adj))
    for i in Adj:
        for j in i:
#             print(j)
            for k in j:
                print(k)
                count1 += 1
    print(count1)
    
    
    nodes_length = len(graph.nodes())
    nx.draw(graph, with_labels=True,node_color = "red")
#     plt.show()
    # Generates chromosomes? basically gives chrom array population value times
    d = {"chrom":[generate_chrom(nodes_length,Adj) for n in range(population)]}
    #makes data frame of data set d
    dframe = pd.DataFrame(data= d)
    dframe["subsets"] = dframe["chrom"].apply(find_subsets)
    dframe["community_score"]=dframe.apply(lambda x: community_score(x["chrom"],x["subsets"],r,Adj),axis=1)
    #Start iterating
    gen = 0
    population_count = population
    while gen < generation:
        for i in range(int(np.floor(population/10))):
            p1 = 0
            p2 = 0
            elites = dframe.sort_values("community_score",ascending=True)[int(np.floor(population/10)):]
            p1 = roulette_selection(elites)
            p2 = roulette_selection(elites)
            child=uniform_crossover(dframe["chrom"][p1],dframe["chrom"][p2],0.8)
            if len(child)==0:
                continue
            child=mutation(child,Adj,0.2)
            child_subsets = find_subsets(child)
            child_cs = community_score(child,child_subsets,r,Adj)
            dframe.loc[population_count]=[child,child_subsets,child_cs]
            population_count += 1
        dfsorted = dframe.sort_values("community_score",ascending=False)
        to_drop = dfsorted.index[population:]
        dframe.drop(to_drop,inplace=True)
        gen +=1  
    sorted_df = dframe.sort_values("community_score",ascending=False).index[0]
    nodes_subsets = dframe["subsets"][sorted_df]
    nodes_list = list(graph.nodes())
    #getting result
    result = []
    for subs in nodes_subsets:
        subset = []
        for n in subs:
            subset.append(nodes_list[n])
        result.append(subset)
    NMI = 0
    clu = dframe.loc[sorted_df]
    clu = clu['chrom']
    clu = np.array(clu)
    for index, target in dframe.iterrows():
        temp = np.array(target['chrom'])
        x = computeNMI(clu,temp)
        NMI += x
    NMI /= len(dframe)
    modularity = nx_comm.modularity(graph, result)
#     print('NMI: ',NMI)
#     print('MODULARITY: ', modularity)
    print_graph(graph, result, nodes)
    return result, NMI, modularity



# returning the the array of column indexes where randomly chosen till value is 1
def generate_chrom(nodes_length,Adj):
    chrom = np.array([],dtype=int)
    for x in range(nodes_length):
        rand = np.random.randint(0,nodes_length)
        while Adj[x,rand] != 1:
            rand = np.random.randint(0,nodes_length)
        chrom = np.append(chrom,rand)
    return chrom

def merge_subsets(sub):
    arr =[]
    to_skip=[]
    for s in range(len(sub)):
        if sub[s] not in to_skip:
            new = sub[s]
            for x in sub:
                if sub[s] & x:
                    new = new | x
                    to_skip.append(x)
            arr.append(new)
    return arr

def find_subsets(chrom):
    sub = [{x,chrom[x]} for x in range(len(chrom))]
    result=sub
    i=0
    while i<len(result):
        candidate = merge_subsets(result)
        if candidate != result:
            result = candidate
        else:
            break
        result=candidate
        i+=1
    return result

def community_score(chrom,subsets,r,Adj):
    matrix = Adj.toarray()
    CS=0
    for s in subsets:
        submatrix = np.zeros((len(chrom),len(chrom)),dtype=int)
        for i in s:
            for j in s:
                submatrix[i][j]=matrix[i][j]
        M=0
        v=0
        PS=0
        for row in list(s):
            ki = np.sum(matrix[row])
            kiin = np.sum(submatrix[row])
            kiout = ki - kiin
            P= kiin/ki
            PS+=P
            row_mean = kiin/len(s)
            v+=np.sum(submatrix[row])
            M+=(row_mean**r)/len(s)
        CS+=M*v
    OS= 0.5*CS/len(subsets) + 0.5*(1/PS)*len(subsets)  #Overall score is calculated by maximizing CS and min PS
    return OS

def roulette_selection(df_elites):
    prob = np.random.random_sample()
    sum_cs=np.sum(df_elites["community_score"])
    x=0
    selected = 0
    for i in df_elites.index:
        x += df_elites["community_score"][i]
        X=x/sum_cs
        if prob < X:
            chosen=i
            break
    return chosen

def uniform_crossover(parent_1,parent_2,crossover_rate):
    if np.random.random_sample() < crossover_rate:
        length = len(parent_1)
        mask = np.random.randint(2, size=length)
        child = np.zeros(length,dtype=int)
        for i in range(len(mask)):
            if mask[i] == 1:
                child[i]=parent_1[i]
            else:
                child[i]=parent_2[i]
        return child
    else:
        return np.array([])

def mutation(chrom,Adj,mutation_rate):
    if np.random.random_sample() < mutation_rate:
        chrom = chrom
        neighbor = []
        while len(neighbor) < 2:
            mutant = np.random.randint(1,len(chrom))
            row = Adj[mutant].toarray()[0]
            neighbor = [i for i in range(len(row)) if row[i]==1]
            if len(neighbor) > 1:
                neighbor.remove(chrom[mutant])
                to_change=int(np.floor(np.random.random_sample()*(len(neighbor))))
                chrom[mutant]=neighbor[to_change]
                neighbor.append(chrom[mutant])
    return chrom

#NMI
def entropy(nums):
    z = np.bincount(nums)
    N = len(nums)
    assert nums.shape == (N, )
    ent = 0.0
    for e in z:
        if e != 0:
            p = float(e) / N
            ent += p*math.log(p)
    assert ent <= 0
    ent = -ent
    assert ent >=0
    return ent

def computeNMI(clusters, classes):
    assert clusters.shape == classes.shape
    A = np.c_[(clusters, classes)]
    A = np.array(A)
    N = A.shape[0]
    assert A.shape == (N, 2)
    H_clusters = entropy(A[:, 0])
    H_classes = entropy(A[:, 1])
    NMI = 0.0
    for k in np.unique(A[:, 0]):
        z = A[A[:, 0] == k, 1]
        len_wk = len(z)
        t = A[:, 1]
        for e in np.unique(z):
            wk_cj=len(z[z==e])
            len_cj=len(t[t == e])
            assert wk_cj <= len_cj
            numerator= (float(wk_cj) / float(N)) * math.log( (N*wk_cj) / float(len_wk * len_cj)  )
            NMI += numerator
    NMI /= float((H_clusters + H_classes) * 0.5)

    assert (NMI > 0.0 or abs(NMI) < 1e-10) and (NMI < 1.0 or abs(NMI - 1.0) < 1e-10)
    return NMI

def print_graph(graph, result, nodes):
    colors = []
    for i in range(0, len(result)):
        colors.append('#%06X' % np.random.randint(0, 0xFFFFFF))
    col_arr = ['']*len(nodes)
    for i in range(0,len(result)):
        for j in result[i]:
            col_arr[j-1] = colors[i]
    nx.draw(graph, with_labels=True,node_color = col_arr)
    plt.show()



















# main start

nodes = []
edges = np.loadtxt('football.txt')
for i in edges:
    for j in i:
        if j not in nodes:
            nodes.append(int(j))

            
#genetic algorithm
NMI_arr_GE = []
mod_arr_GE = []
bst_arr_GE = []

for i in range(0,10):
    best_community, NMI, modularity = genetic_evolution(nodes,edges)
    NMI_arr_GE.append(NMI)
    mod_arr_GE.append(modularity)
    bst_arr_GE.append(best_community)
    
NMI_avg_GE = sum(NMI_arr_GE)/len(NMI_arr_GE)
mod_avg_GE = sum(mod_arr_GE)/len(mod_arr_GE)
NMI_bst_GE = max(NMI_arr_GE)  #best
mod_bst_GE = max(mod_arr_GE)
NMI_lst_GE = min(NMI_arr_GE)  #least
mod_lst_GE = min(mod_arr_GE)

print("Genetic Algorithm")
print('NMI avg: ', NMI_avg_GE)
print('NMI max: ', NMI_bst_GE)
print('NMI min: ', NMI_lst_GE)
print()
print('mod avg: ', mod_avg_GE)
print('mod max: ', mod_bst_GE)
print('mod min: ', mod_lst_GE)


#differential algorithm
NMI_arr_DE = []
mod_arr_DE = []
bst_arr_DE = []

for i in range(0,10):
    best_community, NMI, modularity = differential_evolution(nodes,edges)
    NMI_arr_DE.append(NMI)
    mod_arr_DE.append(modularity)
    bst_arr_DE.append(best_community)
    
NMI_avg_DE = sum(NMI_arr_DE)/len(NMI_arr_DE)
mod_avg_DE = sum(mod_arr_DE)/len(mod_arr_DE)
NMI_bst_DE = max(NMI_arr_DE)  #best
mod_bst_DE = max(mod_arr_DE)
NMI_lst_DE = min(NMI_arr_DE)  #least
mod_lst_DE = min(mod_arr_DE)

print("Differential Algorithm")
print('NMI avg: ', NMI_avg_DE)
print('NMI max: ', NMI_bst_DE)
print('NMI min: ', NMI_lst_DE)
print()
print('mod avg: ', mod_avg_DE)
print('mod max: ', mod_bst_DE)
print('mod min: ', mod_lst_DE)


C:\Users\SHUBH\AppData\Local\Temp\ipykernel_10288\4078165136.py:15: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  Adj = nx.adjacency_matrix(graph)


TypeError: sparse matrix length is ambiguous; use getnnz() or shape[0]